In [1]:
import sys
sys.path.append('../../../config.py')
from config import YOUR_NAME

USER_FLAG = "--user"

In [2]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_text_classification_single_label_sample(
    project: str,
    endpoint_id: str,
    content: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instance = predict.instance.TextClassificationPredictionInstance(
        content=content,
    ).to_value()
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )

    predictions = response.predictions
    return dict(predictions[0])

In [ ]:
PROJECT_ID = "training1emakia"  # @param {type:"string"}
print(PROJECT_ID)

In [ ]:
BUCKET_NAME = f"{YOUR_NAME}_batch_prediction"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
print(BUCKET_NAME)
print(BUCKET_URI)

In [6]:
import pandas as pd
#import jsonlines
import json
from google.cloud import bigquery
from google.cloud import storage

client = bigquery.Client()


In [ ]:
FOLDER_NAME= f"output"
from google.cloud import storage
from itertools import islice
client = storage.Client()

input_file_name = f'{YOUR_NAME}_all_prediction.jsonl'

#gcs_source = 'gs://prediction-data-english/batchpredictioncsv.csv'
gcs_source = f'gs://{YOUR_NAME}_batch_prediction/dataforbatchprediction_csv.csv' 
data = pd.read_csv(
    gcs_source,      # relative python path to subdirectory
    sep=',',           # Tab-separated value file.
    #quotechar="'",        # single quote allowed as quote character
    usecols=['id', 'text']  # Only load the three columns specified.
    #na_values=['.', '??']       # Take any '.' or '??' values as NA
)


bucket = client.bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, row in data.iterrows():
    response = predict_text_classification_single_label_sample(
        project="12807912884",
        endpoint_id="8493069816616189952",
        location="us-central1",
        content=row["text"])
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"instance": {"content": instance_file_uri, "mimeType": "text/plain"},
               "prediction": response,
               "id": row["id"],
               "tweet content": row["text"]}
    input_file_data.append(tmp_data)

from proto.marshal.collections.repeated import RepeatedComposite
    
class CustomEncoder(json.JSONEncoder):
    def default(self, o):
        match o:
            case RepeatedComposite():
                return list(o)
            case _:
                return super().default(o)

input_str = "\n".join(json.dumps(d, cls=CustomEncoder, ensure_ascii=False) for d in input_file_data)
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)
print("jsonl file is created successfully.")

In [ ]:
list("aisdnfiasdnfiowf")
list([1,2,4,2,3,4])
list({"a", "b"})
list({"a": 1, "b": 2})

for i in "aisdnfiasdnfiowf":
    print(i)
    
it = "aisdnfiasdnfiowf".__iter__()
try:
    while True:
        i = it.__next__()
        print(i)
except StopIteration:
    pass